In [1]:
from pipeline.util.platform import start_spark, start_spark_local
from pipeline.util.storage import read_csv, read_parquet, write_parquet, write_repart_parquet, write_delta, write_deltatable
from pyspark.sql import functions as F
from delta import *

In [2]:
spark, loggrer, conf = start_spark(app_name='addessmatch_etl_job')

local


In [22]:
df_abmatch = read_parquet(spark, "{storage_curated}/data/in/addressbase".format(storage_curated = spark.conf.get('storage.curated')))

In [23]:
df_abmatch_id = df_abmatch.withColumn("ADDRESS_ID", F.monotonically_increasing_id())

In [24]:
df_abmatch_id.createOrReplaceTempView("address_in")

In [ ]:
spark.sql("select * from address_in").show(truncate=False)

In [31]:
spark.sql(f"INSERT INTO ADDRESS ( \
          UPRN,OUTCODE,LOCALITY,ADDRESS,ADDRESS_ID )  \
          SELECT \
          UPRN,OUTCODE,LOCALITY,ADDRESS,ADDRESS_ID \
          FROM address_in")

DataFrame[]

In [14]:
#write_delta(spark, df_abmatch, "{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated')))

In [ ]:
# DeltaTable.createOrReplace(spark) \
#     .tableName("address") \
#     .location("{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated'))) \
#     .addColumn("UPRN", "VARCHAR(12)") \
#     .addColumn("OUTCODE", "STRING") \
#     .addColumn("LOCALITY", "STRING") \
#     .addColumn("ADDRESS", "STRING") \
#     .addColumn("ADDRESS_ID", "BIGINT", generatedAlwaysAs=F.monotonically_increasing_id()) \
#     .execute()

In [21]:
loc = "'{storage_curated}/data/out/address'".format(storage_curated = spark.conf.get('storage.curated'))

#spark.sql(f"DROP TABLE ADDRESS")
spark.sql(f"CREATE OR REPLACE TABLE ADDRESS ( \
          UPRN VARCHAR(12), \
          OUTCODE STRING, \
          LOCALITY STRING, \
          ADDRESS STRING, \
          ADDRESS_ID BIGINT )  \
          USING DELTA  \
          LOCATION {loc}")

DataFrame[]

In [5]:
#print(sqlstr)
#sqlstr.replace('\n','')
#print(loc)

'C:/PalProjects/addressmatching/data/out/address'


In [12]:
#write_delta(spark, df_abmatch, "address")
#write_deltatable(spark, df_abmatch, "address")
#df_abmatch.createOrReplaceTempView("address")

In [32]:
#DeltaTable.isDeltaTable(spark, "{storage_curated}/data/out/address".format(storage_curated = spark.conf.get('storage.curated'))) # True
#spark.sql("show tables in default").show(truncate=False)
#spark.sql("describe table extended address_in").show(truncate=False)
spark.sql("select * from ADDRESS").show(truncate=False)
#spark.sql("USE hive_metastore")
#spark.sql("show databases").show(truncate=False)
#spark.sql("show tables").show(truncate=False)
#spark.sql("drop table address")
#print(spark.sharedState.externalCatalog.unwrapped)
#spark.catalog.listTables()
#spark.catalog.currentDatabase()
#spark.sparkContext.getConf().getAll()
#df_abmatch_id.printSchema()
#df_abmatch_id2.show(truncate=False)



+------------+-------+--------+-------------------------------------------------+-----------+
|UPRN        |OUTCODE|LOCALITY|ADDRESS                                          |ADDRESS_ID |
+------------+-------+--------+-------------------------------------------------+-----------+
|100040211947|6LN    |null    |35A ELMSIDE EXETER DEVON EX4 6LN                 |17179869184|
|100040230517|4EU    |null    |38 RADFORD ROAD EXETER DEVON EX2 4EU             |17179869185|
|100040221792|6AP    |null    |80 LONGBROOK STREET EXETER DEVON EX4 6AP         |17179869186|
|100040234411|2NE    |null    |14 ST ALBANS CLOSE EXETER DEVON EX4 2NE          |17179869187|
|100040220123|2DG    |null    |44 KNOWLE DRIVE EXETER DEVON EX4 2DG             |17179869188|
|100040201634|4RN    |null    |15 BAGSHOT AVENUE EXETER DEVON EX2 4RN           |17179869189|
|100040202140|5ED    |null    |27 BARRACK ROAD EXETER DEVON EX2 5ED             |17179869190|
|100040233943|1LR    |null    |44 SOMERSET AVENUE EXETER DEV